# RNN Classification Models
This example shows the application of RNN models in river-torch with and without usage of an incremental class adaption strategy.

In [1]:
from deep_river.classification import RollingClassifier
from river import metrics, compose, preprocessing, datasets
import torch
from tqdm import tqdm

## RNN Model

In [2]:
class RnnModule(torch.nn.Module):

    def __init__(self, n_features, hidden_size=1):
        super().__init__()
        self.n_features = n_features
        self.rnn = torch.nn.RNN(
            input_size=n_features, hidden_size=hidden_size, num_layers=1
        )
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        out, hn = self.rnn(X)  # lstm with input, hidden, and internal state
        hn = hn.view(-1, self.rnn.hidden_size)
        return self.softmax(hn)

### Classification without incremental class adapation strategy

In [3]:
dataset = datasets.Keystroke()
metric = metrics.Accuracy()
optimizer_fn = torch.optim.SGD

model_pipeline = preprocessing.StandardScaler()
model_pipeline |= RollingClassifier(
    module=RnnModule,
    loss_fn="binary_cross_entropy",
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    append_predict=True,
    is_class_incremental=False,
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  RollingClassifier (
    module=None
    loss_fn="binary_cross_entropy"
    optimizer_fn=<class 'torch.optim.sgd.SGD'>
    lr=0.01
    output_is_logit=True
    is_class_incremental=False
    is_feature_incremental=False
    device="cpu"
    seed=42
    window_size=20
    append_predict=True
  )
)

In [4]:
for x, y in tqdm(dataset):
    y_pred = model_pipeline.predict_one(x)  # make a prediction
    metric.update(y, y_pred)  # update the metric
    model_pipeline.learn_one(x, y)  # make the model learn
print(f"Accuracy: {metric.get():.2f}")

0it [00:00, ?it/s]

41it [00:00, 408.88it/s]

95it [00:00, 483.29it/s]

161it [00:00, 562.61it/s]

226it [00:00, 596.24it/s]

...

20215it [00:32, 602.08it/s]

20281it [00:32, 617.53it/s]

20348it [00:32, 631.91it/s]

20400it [00:32, 629.27it/s]

Accuracy: 0.02


### Classification with incremental class adaption strategy

In [5]:
dataset = datasets.Keystroke()
metric = metrics.Accuracy()
optimizer_fn = torch.optim.SGD

model_pipeline = preprocessing.StandardScaler()
model_pipeline |= RollingClassifier(
    module=RnnModule,
    loss_fn="binary_cross_entropy",
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    append_predict=True,
    is_class_incremental=True,
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  RollingClassifier (
    module=None
    loss_fn="binary_cross_entropy"
    optimizer_fn=<class 'torch.optim.sgd.SGD'>
    lr=0.01
    output_is_logit=True
    is_class_incremental=True
    is_feature_incremental=False
    device="cpu"
    seed=42
    window_size=20
    append_predict=True
  )
)

In [6]:
for x, y in tqdm(dataset):
    y_pred = model_pipeline.predict_one(x)  # make a prediction
    metric.update(y, y_pred)  # update the metric
    model_pipeline.learn_one(x, y)  # make the model learn
print(f"Accuracy: {metric.get():.2f}")

0it [00:00, ?it/s]

/Users/cedrickulbach/Documents/Projects/deep-river/deep_river/base.py:231: UserWarning: The model will not be able to adapt its output to new classes since no supported output layer was found.
  warnings.warn(


69it [00:00, 684.00it/s]

138it [00:00, 651.33it/s]

204it [00:00, 646.26it/s]

...

20245it [00:38, 624.27it/s]

20312it [00:38, 635.99it/s]

20378it [00:38, 642.27it/s]

20400it [00:38, 531.22it/s]

Accuracy: 0.02


## LSTM Model

In [7]:
class LstmModule(torch.nn.Module):

    def __init__(self, n_features, hidden_size=1):
        super().__init__()
        self.n_features = n_features
        self.lstm = torch.nn.LSTM(
            input_size=n_features, hidden_size=hidden_size, num_layers=1
        )
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        output, (hn, cn) = self.lstm(
            X
        )  # lstm with input, hidden, and internal state
        hn = hn.view(-1, self.lstm.hidden_size)
        return self.softmax(hn)

### Classifcation without incremental class adaption strategy

In [8]:
dataset = datasets.Keystroke()
metric = metrics.Accuracy()
optimizer_fn = torch.optim.SGD

model_pipeline = preprocessing.StandardScaler()
model_pipeline |= RollingClassifier(
    module=LstmModule,
    loss_fn="binary_cross_entropy",
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    append_predict=True,
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  RollingClassifier (
    module=None
    loss_fn="binary_cross_entropy"
    optimizer_fn=<class 'torch.optim.sgd.SGD'>
    lr=0.01
    output_is_logit=True
    is_class_incremental=False
    is_feature_incremental=False
    device="cpu"
    seed=42
    window_size=20
    append_predict=True
  )
)

In [9]:
for x, y in tqdm(dataset):
    y_pred = model_pipeline.predict_one(x)  # make a prediction
    metric.update(y, y_pred)  # update the metric
    model_pipeline.learn_one(x, y)  # make the model learn
print(f"Accuracy: {metric.get():.2f}")

0it [00:00, ?it/s]

32it [00:00, 319.38it/s]

88it [00:00, 460.14it/s]

154it [00:00, 550.66it/s]

221it [00:00, 595.18it/s]

...

20268it [00:31, 638.18it/s]

20334it [00:31, 642.53it/s]

20399it [00:31, 643.96it/s]

20400it [00:31, 638.51it/s]

Accuracy: 0.02


### Classifcation with incremental class adaption strategy

In [10]:
dataset = datasets.Keystroke()
metric = metrics.Accuracy()
optimizer_fn = torch.optim.SGD

model_pipeline = preprocessing.StandardScaler()
model_pipeline |= RollingClassifier(
    module=LstmModule,
    loss_fn="binary_cross_entropy",
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    append_predict=True,
    is_class_incremental=True,
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  RollingClassifier (
    module=None
    loss_fn="binary_cross_entropy"
    optimizer_fn=<class 'torch.optim.sgd.SGD'>
    lr=0.01
    output_is_logit=True
    is_class_incremental=True
    is_feature_incremental=False
    device="cpu"
    seed=42
    window_size=20
    append_predict=True
  )
)

In [11]:
for x, y in tqdm(dataset):
    y_pred = model_pipeline.predict_one(x)  # make a prediction
    metric.update(y, y_pred)  # update the metric
    model_pipeline.learn_one(x, y)  # make the model learn
print(f"Accuracy: {metric.get()}:.2f")

0it [00:00, ?it/s]

59it [00:00, 584.72it/s]

121it [00:00, 601.33it/s]

186it [00:00, 620.76it/s]

251it [00:00, 628.55it/s]

...

20290it [00:36, 472.13it/s]

20342it [00:36, 482.98it/s]

20394it [00:36, 492.93it/s]

20400it [00:36, 562.37it/s]

Accuracy: 0.05965686274509804:.2f
